In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from plot_utils import plot_images, plot_heatmap

In [ ]:
sdf_calls_path = os.path.join("benchmarks", "performance")

In [ ]:
benchmark_filenames = {}

for dir_name in os.listdir(sdf_calls_path):
    sub_dir_path = os.path.join(sdf_calls_path, dir_name)
    if (os.path.isdir(sub_dir_path)):
        benchmark_filenames[dir_name] = os.listdir(sub_dir_path)

In [ ]:
benchmark_images = {}
time_meassurments = {}

for method, filenames in benchmark_filenames.items():
    folder_path = os.path.join(sdf_calls_path, method)

    benchmark_images[method] = []
    time_meassurments[method] = []

    for filename in filenames:
        file_path = os.path.join(folder_path, filename)
        max_iteration_count = int((os.path.basename(file_path).split(".")[-2]).split("_")[-1])
        
        if (filename.split(".")[-1] == "png"):
            benchmark_image_bgr = cv2.imread(file_path)
            benchmark_images[method].append((max_iteration_count, benchmark_image_bgr))
        elif (filename.split(".")[-1] == "txt"):
            time_meassurment_file = open(file_path, "r+")
            mm = time_meassurment_file.readline().split(" ")
            time_meassurment_file.close()
            time_meassurments[method].append((max_iteration_count, float(mm[0]), float(mm[1])))

    benchmark_images[method].sort(key=lambda x: x[0])
    time_meassurments[method].sort(key=lambda x: x[0])
        
benchmark_images = dict(sorted(benchmark_images.items(), key=lambda x: x[0])[::-1])
time_meassurments = dict(sorted(time_meassurments.items(), key=lambda x: x[0])[::-1])

In [ ]:
benchmark_sdf_call_count_images_gray = {}
avg_sdf_counts_per_pixel = {}
max_iteration_counts = set()

for (method, images) in benchmark_images.items():
    benchmark_sdf_call_count_images_gray[method] = []
    avg_sdf_counts_per_pixel[method] = []

    for (max_iteration_count, image_bgr) in images:
        image_gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
        
        sdf_count_sum = int((cv2.sumElems(image_gray)[0] / 255.0) * max_iteration_count)
        avg_sdf_count_per_pixel = sdf_count_sum / (image_gray.shape[0] * image_gray.shape[1])

        benchmark_sdf_call_count_images_gray[method].append(image_gray)
        avg_sdf_counts_per_pixel[method].append(round(avg_sdf_count_per_pixel, 3))
        max_iteration_counts.add(max_iteration_count)

max_iteration_counts = sorted(max_iteration_counts)

In [ ]:
time_meassurments_chrono = {}
time_meassurments_gl_querry = {}
max_iteration_counts = set()

for (method, meassurments) in time_meassurments.items():
    time_meassurments_chrono[method] = []
    time_meassurments_gl_querry[method] = []
    
    for (max_iteration_count, chrono, gl_querry) in meassurments:
        time_meassurments_chrono[method].append(round(chrono, 3))
        time_meassurments_gl_querry[method].append(round(gl_querry, 3))
        max_iteration_counts.add(max_iteration_count)

max_iteration_counts = sorted(max_iteration_counts)

In [ ]:
plot_images(
    images=np.array(list(benchmark_sdf_call_count_images_gray.values())),
    row_labels=[" ".join(str(i).split("_")) for i in list(benchmark_sdf_call_count_images_gray.keys())],
    col_labels=[str(i) for i in list(max_iteration_counts)],
    figsize=50,
    x_label="Max iteration count",
    y_label="Tracing method",
    title="SDF call counts (divided by max iteration counts) of different tracing methods at different max iteration counts"
)

In [ ]:
plot_heatmap(
    matrix=np.array(list(avg_sdf_counts_per_pixel.values())),
    row_labels=[" ".join(str(i).split("_")) for i in list(avg_sdf_counts_per_pixel.keys())],
    col_labels=[str(i) for i in list(max_iteration_counts)],
    meassurment_unit="",
    x_figsize=15,
    y_figsize=6,
    x_label="Max iteration count",
    y_label="Tracing method",
    title="Average sdf call count per pixel (not including: shadows, ao, normal calculations)"
)

In [ ]:
plot_heatmap(
    matrix=np.array(list(time_meassurments_chrono.values())),
    row_labels=[" ".join(str(i).split("_")) for i in list(time_meassurments_chrono.keys())],
    col_labels=[str(i) for i in list(max_iteration_counts)],
    meassurment_unit="ms",
    x_figsize=15,
    y_figsize=6,
    x_label="Max iteration count",
    y_label="Tracing method",
    title="Averaged render times in ms (chrono)"
)

In [ ]:
plot_heatmap(
    matrix=np.array(list(time_meassurments_gl_querry.values())),
    row_labels=[" ".join(str(i).split("_")) for i in list(time_meassurments_gl_querry.keys())],
    col_labels=[str(i) for i in list(max_iteration_counts)],
    meassurment_unit="ms",
    x_figsize=15,
    y_figsize=6,
    x_label="Max iteration count",
    y_label="Tracing method",
    title="Averaged render times in ms (gl querry)"
)